In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
image_size = 96
image_color = 1
batch_size = 5
training_file_name = '/home/syso/Tensorflow/prac/training.csv'
test_file_name = '/home/syso/Tensorflow/prac/test.csv'

In [ ]:
def get_input_queue(csv_file_name, num_epochs = None):
    print("get_input_queue")
    csv_file = pd.read_csv(csv_file_name,sep=',',header=None)
    csv_file = np.array(csv_file)
    train_images = []
    train_labels = []
    for line in csv_file:
        train_images.append(line[0])
        train_labels.append(int(line[1]))
    input_queue = tf.train.slice_input_producer([train_images, train_labels], shuffle = True)
    
    return input_queue

def read_data(input_queue):
    print("read_data")
    image_file = input_queue[0]
    label = input_queue[1]
    image = tf.image.decode_jpeg(tf.read_file(image_file),channels=1)
    return image,label,image_file


def read_data_batch(csv_file_name, batch_size = batch_size):
    print("read_data_batch")
    input_queue = get_input_queue(csv_file_name)
    image,label,file_name= read_data(input_queue)
    image = tf.reshape(image,[image_size*image_size*image_color])
    
    batch_image,batch_label,batch_file = tf.train.batch([image,label,file_name],batch_size=batch_size)
    batch_file = tf.reshape(batch_file,[batch_size,1])
    
    print("read_data_batch")
    
    return batch_image,batch_label,batch_file

In [ ]:
n1 = 16
n2 = 32
n3 = 64
ksize = 5

weights={
    'ce1': tf.Variable(tf.random_normal([ksize,ksize,image_color,n1], stddev=0.1)),
    'ce2': tf.Variable(tf.random_normal([ksize,ksize,n1,n2], stddev=0.1)),
    'ce3': tf.Variable(tf.random_normal([ksize,ksize,n2,n3], stddev=0.1)),
    'cd3': tf.Variable(tf.random_normal([ksize,ksize,n2,n3], stddev=0.1)),
    'cd2': tf.Variable(tf.random_normal([ksize,ksize,n1,n2], stddev=0.1)),
    'cd1': tf.Variable(tf.random_normal([ksize,ksize,image_color,n1], stddev=0.1))
    
}

biases = {
    'be1': tf.Variable(tf.random_normal([n1], stddev=0.1)),
    'be2': tf.Variable(tf.random_normal([n2], stddev=0.1)),
    'be3': tf.Variable(tf.random_normal([n3], stddev=0.1)),
    'bd3': tf.Variable(tf.random_normal([n2], stddev=0.1)),
    'bd2': tf.Variable(tf.random_normal([n1], stddev=0.1)),
    'bd1': tf.Variable(tf.random_normal([1], stddev=0.1))
    
}

In [ ]:
def cae(_X, _W, _b, _keepprob):
    _input_r = tf.reshape(_X,shape=[-1, image_size,image_size,image_color])
    #encode
    _ce1 = tf.nn.sigmoid(tf.add(tf.nn.conv2d(_input_r,_W['ce1'],strides=[1,2,2,1], padding='SAME'),_b['be1']))
    _ce1 = tf.nn.dropout(_ce1,_keepprob)
    _ce2 = tf.nn.sigmoid(tf.add(tf.nn.conv2d(_ce1,_W['ce2'],strides=[1,2,2,1], padding='SAME'),_b['be2']))
    _ce2 = tf.nn.dropout(_ce2, _keepprob)
    _ce3 = tf.nn.sigmoid(tf.add(tf.nn.conv2d(_ce2,_W['ce3'],strides=[1,2,2,1], padding='SAME'),_b['be3']))
    _ce1 = tf.nn.dropout(_ce3,_keepprob)
    
    #decode
    _cd3 = tf.nn.sigmoid(tf.add(tf.nn.conv2d_transpose(_ce3,_W['cd3'],tf.stack([tf.shape(_X)[0],24, 24, n2]), strides=[1, 2, 2, 1],
                               padding='SAME'),_b['bd3']))
    _cd3 = tf.nn.dropout(_cd3, _keepprob)
    _cd2 = tf.nn.sigmoid(tf.add(tf.nn.conv2d_transpose(_cd3,_W['cd2'], tf.stack([tf.shape(_X)[0], 48, 48, n1]), strides=[1,2,2,1],
                                                      padding='SAME'), _b['bd2']))
    _cd2 = tf.nn.dropout(_cd2, _keepprob)
    _cd1 = tf.nn.sigmoid(tf.add(tf.nn.conv2d_transpose(_cd2,_W['cd1'], tf.stack([tf.shape(_X)[0],96,96,1]), strides=[1,2,2,1],
                                                      padding='SAME'), _b['bd1']))
    _cd1 = tf.nn.dropout(_cd1, _keepprob)
    _out = _cd1
    return {'input_r': _input_r, 'ce1':_ce1, 'ce2': _ce2, 'ce3': _ce3, 'cd3':_cd3, 'cd2': _cd2, 'cd1':_cd1,
            'layers':(_input_r, _ce1, _ce2, _ce3, _cd3, _cd2, _cd1), 'out':_out}

In [ ]:
x = tf.placeholder(tf.float32, [None, image_size*image_size])
y = tf.placeholder(tf.float32, [None, image_size*image_size])
print(x)
keepprob = tf.placeholder(tf.float32)
pred = cae(x, weights, biases, keepprob)

cost = tf.reduce_mean(tf.square(cae(x, weights, biases, keepprob)['out']-tf.reshape(y,shape=[-1,image_size,image_size, image_color])))
cost = tf.reduce_mean(tf.square(tf.reshape(cae(x, weights, biases, keepprob)['out'], shape=[-1,1]) - tf.reshape(y,shape=[-1,1])))

learning_rate = 0.001
optm = tf.train.AdamOptimizer(learning_rate).minimize(cost)
init = tf.global_variables_initializer()

In [ ]:
sess = tf.Session()
sess.run(init)
mean_img = np.zeros((image_size*image_size))
batch_xs, _, _ = read_data_batch(training_file_name)
test_xs, _, _ = read_data_batch(test_file_name)
n_epochs   = 5000
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
for epoch_i in range(n_epochs):

    for batch_i in range(8//batch_size):
        batch_xs_ = sess.run(batch_xs)
        print('batch_xs')
        trainbatch = np.array([img - mean_img for img in batch_xs_])
        #print(trainbatch[0])
        #plt.imshow(trainbatch[0].reshape(96,96), cmap='Greys', interpolation='nearest')
        #plt.show()
        trainbatch_noisy = trainbatch - 10*np.random.randn(trainbatch.shape[0], image_size*image_size)
        #print(trainbatch[0])
        #plt.imshow(trainbatch_noisy[0].reshape(96,96), cmap='Greys', interpolation='nearest')
        #plt.show()
        print('run...')
        sess.run(optm, feed_dict={x:trainbatch_noisy, y:trainbatch, keepprob:0.7})
        
        print ("[%02d/%02d] cost: %.4f" % (epoch_i, n_epochs, sess.run(cost, feed_dict={x: trainbatch_noisy, y: trainbatch, keepprob: 1.})))
        
        if (epoch_i % 1) == 0:
                        
            test_xs_ = sess.run(test_xs)
            print('test_xs')
            test_xs_noisy = test_xs_ + 10*np.random.randn(test_xs_.shape[0], image_size*image_size)
            recon = sess.run(pred, feed_dict={x: test_xs_noisy, keepprob: 1.})
            #print(recon)            
            #plt.subplots(2, 2)
            _, axs = plt.subplots(2,2)
            axs[0][0].matshow(np.reshape(trainbatch[0],(96,96)), cmap=plt.get_cmap('gray'))
            axs[0][1].matshow(np.reshape(test_xs_noisy[0],(96,96)), cmap=plt.get_cmap('gray'))
            axs[1][0].matshow(np.reshape(trainbatch_noisy[0],(96,96)), cmap=plt.get_cmap('gray'))
            axs[1][1].matshow(np.reshape(recon['out'][0],(96,96)), cmap=plt.get_cmap('gray'))
            #plt.imshow(test_xs_noisy[0].reshape(96,96).reshape(96,96), cmap='Greys', interpolation='nearest')
            #plt.show()
            #plt.imshow(recon['out'][0].reshape(96,96).reshape(96,96), cmap='Greys', interpolation='nearest')
            plt.show()
                
            
            
coord.request_stop()
coord.join(threads)

In [ ]:
test_xs_ = sess.run(test_xs)

tt = sess.run(pred, feed_dict={x: test_xs_, keepprob: 1.})
plt.imshow(tt['out'][0].reshape(96,96).reshape(96,96), cmap=plt.get_cmap('gray'), interpolation='nearest')
plt.show()

In [ ]:
layers = tt['layers']
for i in range(len(layers)):
    currl = layers[i]
    print (("Shape of layer %d is %s") % (i+1, currl.shape,))